# Data Modeling With Apache Cassandra

#### This project, part of the [Udacity Data Engineering Nanodegree](https://www.udacity.com/course/data-engineer-nanodegree--nd027), consists of the creation of an ETL Data Pipeline using music streaming data files and Apache Cassandra. The purpose of this project is to model a Cassandra database structure using data from raw .csv files that will allow for the 3 queries outlined in Part II below.  

# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages

In [44]:
import cassandra
import os
import glob
import csv
import pandas as pd

#### Here we are reading .csv files from the *event_data* directory into a list of file paths.

In [45]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    # rewrote this because file_path_list was getting overwritten
    # with the files in .ipynb_checkpoints - jrw
    if not root.endswith('.ipynb_checkpoints'):
        file_path_list = glob.glob(os.path.join(root,'*'))
        # Uncomment to view a list of the files
        # print(file_path_list)

/Users/joe/Desktop/Udacity_Data_Engineering/Part2_DataModeling/Lesson5_Project_DataModelingWithApacheCassandra


#### Here we are extracting data from the above files to create a single file of the data we need for this exercise. The output is being written to *event_datafile_new.csv*.

In [46]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            # print(line)
            full_data_rows_list.append(line)
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [47]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Creating Tables and Queries in Apache Cassandra

#### Now we are going to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [48]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating a Keyspace

In [49]:
try:
    session.execute(
        """
        CREATE KEYSPACE IF NOT EXISTS music
        WITH REPLICATION =
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
        """
    )
except Exception as e:
    print(e)

#### Setting the Keyspace

In [50]:
session.set_keyspace("music")

### Tables in the Code Below Are Designed to Allow for the Following Queries:

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1
The requirement for Query 1 is to fetch the artist, song title and song length using a combination of sessionId and itemInSession values. In this case, we are using the session_id field as the partition key and a combination of the session_id and item_in_session fields as the Composite Key for each row. The results of the query will return artist, song title and song length for a single itemInSession value within a sessionId.

#### Create The Table

In [51]:
try:
    session.execute(
        """
        CREATE TABLE IF NOT EXISTS session_plays
            (
                session_id INT,
                item_in_session INT,
                artist TEXT,
                song TEXT, 
                length FLOAT,
                PRIMARY KEY (session_id, item_in_session)
            )
        """
    )
except Exception as e:
    print(e)
                    

#### Insert Values into the Table

In [52]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_plays (session_id, item_in_session, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        insert_vals =[int(line[8]), int(line[3]), line[0], line[9], float(line[5])]
        session.execute(query, insert_vals)
print("Finished Inserting Values")


Finished Inserting Values


#### Query the Table according to Query 1

In [53]:
query = "SELECT artist, song, length from session_plays WHERE session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
if not rows:
    print("Not Found")
else:
    df = pd.DataFrame(list(rows))
    print(df)


      artist                             song      length
0  Faithless  Music Matters (Mark Knight Dub)  495.307312


### Query 2
The requirement for Query 2 is to fetch the artist name, song, and first and last name of the user for all of the songs that a user played in a particular session. The where clause values for this query are *userId* and *sessionId*, and the results must be sorted ascending by *itemInSession* values. Based on these requirements, we are using a combination of `user_id` and `session_id` for the partition key, and `item_in_session` for the clustering key. This will allow for faster querying since `user_id` and `session_id` will be partitioned together, as well as the results be sorted ascending according to the values in the `item_in_session` field.

#### Create the Table

In [54]:
try:
    session.execute(
        """
        CREATE TABLE IF NOT EXISTS user_session_plays
            (
                user_id INT,
                session_id INT,
                item_in_session INT,
                artist TEXT,
                song TEXT,
                first_name TEXT,
                last_name TEXT,
                PRIMARY KEY ((user_id, session_id), item_in_session)
            )
        """
    )
except Exception as e:
    print(e)


#### Insert Values into the Table

In [55]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_session_plays (user_id, session_id, item_in_session, artist, song, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        insert_vals =[int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]]
        session.execute(query, insert_vals)
print("Finished Inserting Values")


Finished Inserting Values


#### Query the Table According to Query 2

In [57]:
query = "SELECT artist, song, first_name, last_name from user_session_plays WHERE user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
if not rows:
    print("No Results Found")
else:
    df = pd.DataFrame(list(rows))
    print(df)


              artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  first_name last_name  
0     Sylvie      Cruz  
1     Sylvie      Cruz  
2     Sylvie      Cruz  
3     Sylvie      Cruz  


### Query 3
The requirement for Query 3 is to fetch the first and last names of users that have ever played a particular song title. The where clause in this case will use a string that is the title of the song. In this case, we will use a Composite Key of *song* and *userId* for defining unique rows. We will also store the first and last names of the users according to each unique *userId*.   

#### Create the Table

In [58]:
try:
    session.execute(
        """
        CREATE TABLE IF NOT EXISTS user_song_plays
            (
                song TEXT,
                user_id INT,
                first_name TEXT,
                last_name TEXT,
                PRIMARY KEY (song, user_id)
            )
        """
    )
except Exception as e:
    print(e)


#### Insert Data into the Table

In [59]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_song_plays (song, user_id, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"
        insert_vals =[line[9], int(line[10]), line[1], line[4]]
        session.execute(query, insert_vals)
print("Finished Inserting Values")


Finished Inserting Values


#### Query the Table According to Query 3

In [61]:
query = "SELECT first_name, last_name from user_song_plays WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
if not rows:
    print("No Results Found")
else:
    df = pd.DataFrame(list(rows))
    print(df)


   first_name last_name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson


### Drop the tables before closing out the sessions

In [62]:
session.execute("DROP TABLE IF EXISTS session_plays")
session.execute("DROP TABLE IF EXISTS user_session_plays")
session.execute("DROP TABLE IF EXISTS user_song_plays")

### Close the session and cluster connection¶

In [63]:
session.shutdown()
cluster.shutdown()